<a href="https://colab.research.google.com/github/Nakul24-1/Viz-for-ML/blob/main/D3_in_Jupyter/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1: CS-GY 9223
## Exploring 20 NewsGroups

In this homework, you will write a D3 Visualization in Javascript and integrate it in Jupyter Notebook.

The goal of this exercise is to explore the **20 News Groups dataset**, a popular machine learning dataset that contains news articles grouped in 20 topics. Your visualization should receive the dataset and display a bar chart with the top most frequent words in the dataset. The user should be able to filter the data based on topic (for example, by clicking in checkboxes, selecting from a drop down menu, etc.). The user should also be able to export the selected documents from the selected topic back to Python (using a button).

In summary, your visualization should have the following capabilities:
- Display a bar chart with the top K words in the document collection
- Enable the user to filter the documents based on topic, and display a bar chart with the frequency of the top K words from that topic.
- Export the documents from the selected news topic back to python (as a list of strings).
- The visualization has to be integrated with python. The API should have two functions:
  - `plot_top_words(documents, K) # plot top K words using D3 and Javascript`
  - `get_exported_documents() # get the exported documents back to python`
  
Example of the resulting visualization:
<img src="HW_Vis.png" width = "500px" height="100px"/>

### Accessing the data

The data should be accessed from sklearn. In this section we show an example of code for accessing the documents and the document classes.

In [ ]:
# Fetching the data
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

These are the 20 topics in the dataset:

In [ ]:
topic_names

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

The documents and document topics are assigned to the variables *documents* and *topics*. We print some document examples below.

In [ ]:
for i in range(2):
    print("Topic: {}".format(topics[i]))
    print("-"*60)
    print("Document:")
    print(documents[i])
    print("="*60)
    print("")

Topic: rec.autos
------------------------------------------------------------
Document:
I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

Topic: comp.windows.x
------------------------------------------------------------
Document:
I'm not familiar at all with the format of these "X-Face:" thingies, but
after seeing them in some folks' headers, I've *got* to *see* them (and
maybe make one of my own)!

I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces")
and I've managed to compile [un]compface too... but now that I'm *looking*
for them, I can't seem to find any

In [ ]:
from collections import Counter

In [ ]:
import glob
import random
import json
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop = set(stopwords.words('english'))
all_stops = stop | set(string.punctuation)
all_stops.add('--')


In [ ]:
wordlist = []
selected_topics= topic_names  # vary this as per the filter

for x in range (len(documents)):
  if topics[x] in selected_topics:
    for i in documents[x].split():
      if i.lower() not in all_stops:
        wordlist.append(i)
dic = Counter(wordlist)

In [ ]:
for y in dic.values():
  for x in range (1,6):
    print (y)
  if x == 5 :
    break;

526
526
526
526
526


In [ ]:
dic.most_common(7)

[('would', 3870),
 ('one', 2930),
 ('like', 2247),
 ('0', 2150),
 ('get', 2144),
 ('know', 1906),
 ('people', 1880)]

In [ ]:
array = []
word = []
dict1 = []
n = 8
for i in range(n):
  dict1.append((dic.most_common(n)[i][1],(dic.most_common(n)[i][0])))
  array.append(dic.most_common(n)[i][1])
  word.append((dic.most_common(n)[i][0]))

data_dict = {
    "array": array,
    "word": word
    
}


In [ ]:
from IPython.display import display, HTML
import json
data_dict

{'array': [3870, 2930, 2247, 2150, 2144, 1906, 1880, 1789],
 'word': ['would', 'one', 'like', '0', 'get', 'know', 'people', 'think']}

CODE V1

In [ ]:
def target_func(comm, msg):
   
  # comm is the kernel Comm instance

  # Register handler for later messages
  #@comm.on_msg
  #def _recv(msg):
  # Use msg['content']['data'] for the data in the message
  counter1 = 0;
  val1 = msg['content']['data'][0]
  
  val2 = msg['content']['data'][1]
  #print(val2)
  selected_topics= [val1]  # vary this as per the filter
  wordlist = []

  for x in range (len(documents)):
    if topics[x] in selected_topics:
      if counter1 < 6:
        print(documents[x])
        counter1= counter1 + 1
      for i in documents[x].split():
        if i.lower() not in all_stops:
          wordlist.append(i)
    
  dic = Counter(wordlist)
  array = []
  word = []
  dict1 = []
  n = int(val2)
  print(val1)
  print(n)
  for i in range(n):
    dict1.append((dic.most_common(n)[i][1],(dic.most_common(n)[i][0])))
    array.append(dic.most_common(n)[i][1])
    word.append((dic.most_common(n)[i][0]))

  data_dict = {
      "array": array,
      "word": word
      
}


  comm.send(data_dict,None)

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

#get_ipython().kernel.comm_manager.register_target('comm_target', target_func)


In [ ]:
from IPython.display import HTML
from IPython.display import Javascript

display(HTML(
    """

<html>
  <head>
    <script src="https://d3js.org/d3.v6.min.js"></script>
</head>

  <body>
    <div id="div_receive_data">
      <p>
  <label># words</label>
  <input type="number" min="1" max="30" step="1" value="8" id="nBin">
</p>
      </div>
    <div id = "dropdown"/>
<script>

 {{

function getArray(object) {{
    return Object.keys(object).reduce(function (r, k) {{
        object[k].forEach(function (a, i) {{
            r[i] = r[i] || {{}};
            r[i][k] = a;
        }});
        return r;
    }}, []);
}}

function getObject(array) {{
    return array.reduce(function (r, o, i) {{
        Object.keys(o).forEach(function (k) {{
            r[k] = r[k] || [];
            r[k][i] = o[k];
        }});
        return r;
    }}, {{}});
}}

function bar_chart(data)
{{

  const data2 = getArray(data)
    
  var result = Object.entries(data);
  
  var word = data.word
  
  const margin = ({{top: 20, bottom: 45, left: 75, right: 30}})  
  const totalWidth = 1000
  const totalHeight = 500
  const visWidth = totalWidth - margin.left - margin.right
  const visHeight = totalHeight - margin.top - margin.bottom
  var y = d3.scaleBand()
    .domain(word)
    .range([0, visHeight])
    .padding(0.2)
  var x = d3.scaleLinear()
    .domain([0, d3.max(data.array)])
    .range([0, visWidth])

  const yAxis = d3.axisLeft(y);
  const xAxis = d3.axisBottom(x);

  //const svg = d3.create('svg')
    const svg = d3.select("#div_receive_data").append("svg")
      .attr('width', totalWidth)
      // we could also put visHeight + margin.top + margin.bottom instead of totalHeight
      .attr('height', totalHeight);
  

  const g = svg.append("g")
      .attr('transform', `translate(${{margin.left}}, ${{margin.top}})`);

  
  g.selectAll('rect')
    .data(data2)
    .join('rect')
      // set attributes for each bar
      .attr('x', 0) // each bar starts at the same x position
      .attr('y', d => y(d.word)) // pass the name to the y-scale to get y position
      .attr('width', d => x(d.array)) // pass the score to the x-scale to get width of the bar
      .attr('height', y.bandwidth()) // get the width of each band in the scale
      .attr('fill', 'steelblue');



  g.append('g')
      .call(yAxis)
        
  // add a group for the x-axis
  g.append('g')
      // we have to move this group down to the bottom of the vis
      .attr('transform', `translate(0, ${{visHeight}})`)
      .call(xAxis)
    // add a label for the x-axis
    .append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('x', visWidth / 2)
      .attr('y', 40)
      .text("Count");

}}
bar_chart({data})

     var allGroup = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc']

    // Initialize the button

    var wordc = 10
    var dropdownButton = d3.select("#dropdown")
      .append('select').attr("value", function (d) {{ return d; }})
      
       var num_words = d3.select("#nBin").on("input", function() {{
  wordc = this.value;
  }});

      dropdownButton // Add a button
  .selectAll('myOptions')
 	.data(allGroup)
  .enter()
	.append('option')
  .text(function (d) {{ return d; }}) // text showed in the menu
  .attr("value", function (d) {{ return d; }}) // corresponding value returned by the button


  dropdownButton.on("change", function(d) {{

    // recover the option that has been chosen
    var selectedOption = d3.select(this).property("value");
    console.log(selectedOption);
    console.log(wordc);
  
    (async () => {{
    const channel = await google.colab.kernel.comms.open('my_comm_target', [selectedOption,wordc]);
    let success = false;

    for await (const message of channel.messages) {{

      d3.select("#div_receive_data").selectAll("svg").remove()
      let data = message.data;
      console.log(data)
      bar_chart(data) 


    
    }} ;

    }})()
    
  }})
 
  
}}

</script>
</body>
</html>
""".format(data = json.dumps(data_dict))))



It is said that CELP vocoders can run on the highest speed 486s with
some room to spare -- they turn 64kbit (8 bit samples, 8k samples/sec)
into 4800 baud. However, DSP is hairy, and I have yet to see actual
proof of this in the form of an implementation. I have heard fairly
reliable rumors to the effect that a famous internetworking guru has a
CELP implementation that runs on Sparcstation 1+'s with some room to
spare, but I have not succeeded thus far in getting my hands on a copy
-- the guru in question has a reputation for not releasing code
without having beaten on it for a very very long time first. 

DSP experts are heavily encouraged to try their own hand at this
problem.

--
Perry Metzger		pmetzger@shearson.com
Tom Clancy omitted these key steps to try to prevent groups of people from
building a nuclear bomb.  However, he asserts that you can find these key
steps in any university library.  The main point of _Five Minutes To Midnight_
is that it is impossible to prevent the pr

In [ ]:
topic_names

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

In [ ]:
get_ipython().kernel.comm_manager.comms

{}

In [ ]:
def setup_comm_colab(api_call_id, callback):
    """Function that connects javascript call with a Colab Notebook"""
    from google.colab import output
    from IPython import display
    def _recv(msg):
        return display.JSON(callback(msg)) # Use display.JSON to transfer an object
    output.register_callback(api_call_id, _recv)
